# golub dataset

In [ ]:
# install.packages("binom") # Installed from R studio as needs compilation
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("cancerclass")

In [1]:
library("binom") 
library("evaluomeR")
library("cancerclass")
library("dplyr")

options(scipen=10)

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which

In [ ]:
# https://rdrr.io/bioc/cancerclass/man/GOLUB.html

In [2]:
data(GOLUB1)
golub = as.data.frame(GOLUB1)

In [3]:
head(golub)
print(paste0("Rows: ", nrow(golub)))
print(paste0("Columns: ", ncol(golub)))
# En el artículo dice: The data are then summarized by a 72 × 3,571

,AFFX.BioDn.3_at,AFFX.BioB.5_st,AFFX.HUMISGF3A.M97935_MA_at,AFFX.HUMISGF3A.M97935_MB_at,AFFX.HUMISGF3A.M97935_3_at,AFFX.HUMRGE.M10098_5_at,AFFX.HUMRGE.M10098_M_at,AFFX.HUMRGE.M10098_3_at,AFFX.HUMGAPDH.M33197_5_at,AFFX.HUMGAPDH.M33197_M_at,...,U58516_at,X06956_at,X83863_at,Z17240_at,M71243_f_at,class,sample,type,FAB,gender
X1,199,206,-13,215,797,14538,9738,8529,15076,11126,...,511,389,793,329,191,ALL,BM,B-cell,,M
X2,-330,74,-219,116,433,615,115,1518,19448,13568,...,837,442,782,295,76,ALL,BM,T-cell,,M
X3,33,-215,104,476,1474,5669,3272,3668,27410,16756,...,1199,168,1138,777,228,ALL,BM,T-cell,,M
X4,158,31,-148,155,415,4850,2293,2569,14920,11439,...,835,174,627,170,126,ALL,BM,B-cell,,
X5,4,252,-55,122,483,1284,2731,316,14653,15030,...,649,504,250,314,56,ALL,BM,B-cell,,
X6,67,193,-327,176,412,4148,2743,2404,25771,24420,...,1221,172,645,341,193,ALL,BM,T-cell,,M


[1] "Rows: 72"
[1] "Columns: 3576"


In [10]:
golub["class"] = NULL
golub["sample"] = NULL
golub["type"] = NULL
golub["FAB"] = NULL
golub["gender"] = NULL
print(paste0("Rows: ", nrow(golub)))
print(paste0("Columns: ", ncol(golub)))
golub["labels"] = rownames(golub)
golub = golub[ , c("labels", names(golub)[names(golub) != "labels"])] 

[1] "Rows: 72"
[1] "Columns: 3572"


In [11]:
head(golub)

,labels,AFFX.BioDn.3_at,AFFX.BioB.5_st,AFFX.HUMISGF3A.M97935_MA_at,AFFX.HUMISGF3A.M97935_MB_at,AFFX.HUMISGF3A.M97935_3_at,AFFX.HUMRGE.M10098_5_at,AFFX.HUMRGE.M10098_M_at,AFFX.HUMRGE.M10098_3_at,AFFX.HUMGAPDH.M33197_5_at,...,U88902_cds1_f_at,D64015_at,L78833_cds4_at,M93143_at,U29175_at,U58516_at,X06956_at,X83863_at,Z17240_at,M71243_f_at
X1,X1,199,206,-13,215,797,14538,9738,8529,15076,...,346,229,349,384,1582,511,389,793,329,191
X2,X2,-330,74,-219,116,433,615,115,1518,19448,...,290,194,214,231,624,837,442,782,295,76
X3,X3,33,-215,104,476,1474,5669,3272,3668,27410,...,220,294,464,720,753,1199,168,1138,777,228
X4,X4,158,31,-148,155,415,4850,2293,2569,14920,...,430,128,342,307,743,835,174,627,170,126
X5,X5,4,252,-55,122,483,1284,2731,316,14653,...,159,71,159,178,626,649,504,250,314,56
X6,X6,67,193,-327,176,412,4148,2743,2404,25771,...,199,168,147,384,1157,1221,172,645,341,193


In [12]:
variance = sort(sapply(golub[-1], var), decreasing = TRUE)  # Sorted gene variance
gene_var = as.data.frame(variance)
gene_var["gene"] = rownames(gene_var)

In [ ]:
top_number = 100
top_genes = gene_var[c(1:top_number), ]
head(top_genes)

In [ ]:
gene_list = as.list(top_genes["gene"])

In [ ]:
top_gene_list = unlist(setdiff(gene_list, names(golub)))
top_gene_list

In [ ]:
top_golub = golub[, top_gene_list]
top_golub["labels"] = rownames(top_golub)
top_golub = golub[ , c("labels", names(top_golub)[names(top_golub) != "labels"])] 

In [ ]:
head(top_golub)

# evaluomeR K analysis

In [ ]:
seed = 13606
k.range=c(2,6)
cbi = "kmeans"

golub_stab_range = stabilityRange(data=top_golub, k.range=k.range, 
                            bs=500, seed=seed,
                            all_metrics=TRUE,
                            cbi=cbi)
golub_stab = standardizeStabilityData(golub_stab_range)

In [ ]:
golub_qual_range = qualityRange(data=top_golub, k.range=k.range, 
                            seed=seed,
                            all_metrics=TRUE,
                            cbi=cbi)
golub_qual = standardizeQualityData(golub_qual_range)

In [ ]:
print("W/ kmeans:")
golub_stab
golub_qual

In [ ]:
k.range.opt = c(3,6)
golub_k_opt = getOptimalKValue(golub_stab_range, golub_qual_range, k.range= k.range.opt)